In [ ]:
!gmx --version

In [ ]:
!mkdir sim
%cd sim

In [ ]:
%%bash

git clone https://github.com/m3g/packmol.git
cd packmol
git checkout v20.3.5
./configure
make

In [ ]:
!git clone https://github.com/zhmurov/trappeua
!mv trappeua/trappeua.ff .
!rm -rf trappeua

In [ ]:
!packmol/packmol < input/packmol.inp

In [ ]:
!cp input/topol.top .

In [ ]:
!gmx editconf -f conf.pdb -o conf.gro -box 4 4 4
!gmx grompp -f input/em.mdp -c conf.gro -o em.tpr
!gmx mdrun -deffnm em -nt 4

In [ ]:
!gmx grompp -f input/nvt.mdp -c em.gro -o nvt.tpr
!gmx mdrun -deffnm nvt -nt 4

In [ ]:
!gmx grompp -f input/npt.mdp -c nvt.gro -o npt.tpr
!gmx mdrun -deffnm npt -nt 4

In [ ]:
%%bash

ELECTRIC_FIELD=1.0
cp input/md_iso_E.mdp md_iso_${ELECTRIC_FIELD}.mdp
sed -i "s/ELECTRIC_FIELD/${ELECTRIC_FIELD}/g" md_iso_${ELECTRIC_FIELD}.mdp

In [ ]:
!gmx grompp -f md_iso_${ELECTRIC_FIELD}.mdp -c npt.gro -o md_iso_${ELECTRIC_FIELD}.tpr
!gmx mdrun -deffnm md_iso_${ELECTRIC_FIELD} -nt 4

In [ ]:
import MDAnalysis as mda

#electric_fields = ["0.00", "0.01", "0.1", "1.0", "10.0", "50.0"]
electric_fields = ["0.2", "0.5"]

for electric_field in electric_fields:

    u = mda.Universe("data/long/conf.pdb", "data/long/md_iso_" + electric_field + ".xtc", in_memory="True")
    step = 0
    order_params = []
    times  = []
    for ts in u.trajectory:
        box = ts.dimensions
        cosxi2 = 0.0
        ncosxi2 = 0
        ends1 = u.select_atoms("name N1")
        ends2 = u.select_atoms("name C19")
        for i in range(0, len(ends1) - 1):
            endi1 = ends1[i].position
            endi2 = ends2[i].position
            ri = endi2 - endi1
            ri[0] = ri[0] - round(ri[0] / box[0]) * box[0]
            ri[1] = ri[1] - round(ri[1] / box[1]) * box[1]
            ri[2] = ri[2] - round(ri[2] / box[2]) * box[2]
            ri_len = (ri[0]*ri[0] + ri[1]*ri[1] + ri[2]*ri[2]) ** 0.5
            for j in range(i + 1, len(ends1)):
                endj1 = ends1[j].position
                endj2 = ends2[j].position
                rj = endj2 - endj1
                rj[0] = rj[0] - round(rj[0] / box[0]) * box[0]
                rj[1] = rj[1] - round(rj[1] / box[1]) * box[1]
                rj[2] = rj[2] - round(rj[2] / box[2]) * box[2]
                rj_len = (rj[0]*rj[0] + rj[1]*rj[1] + rj[2]*rj[2]) ** 0.5
                cosxi = (ri[0]*rj[0] + ri[1]*rj[1] + ri[2]*rj[2]) / (ri_len * rj_len)
                cosxi2 += cosxi*cosxi
                ncosxi2 += 1
        cosxi2 /= ncosxi2
        order_param = (3.0*cosxi2 - 1.0)/2.0
        order_params.append(order_param)
        times.append(step)
        step += 1
        print(step, order_param)

    import matplotlib.pyplot as plt
    fig = plt.figure()
    plt.plot(times, order_params)
    plt.show()
    fig.savefig("order_parameter_" + electric_field + ".png")